In [3]:
# load in libraries

import pandas as pd
import pandas_gbq
from google.cloud import bigquery
from google.oauth2 import service_account
from time import sleep
from newspaper import Article

In [5]:
# import csv file to filter sources by country 

country_filter = pd.read_csv('/Users/KevinLubin/Desktop/ds/pearl/gdelt_country_sources.csv')
country_filter.head()

,source,fips
0,0-100.it,IT
1,0-50.ru,RS
2,0-60mag.com,US
3,0-debt.com,US
4,000fff.org,US


In [6]:
# define credentials object for GCP to run queries

credentials = service_account.Credentials.from_service_account_file(
    '/Users/KevinLubin/Desktop/ds/pearl/pearl-336700-0fa91569420d.json')

In [58]:
# Perform query

query = """

    SELECT GlobalEventID, SQLDATE, EventCode, EventBaseCode, EventRootCode, Quadclass, AvgTone,
    GoldSteinScale, NumMentions, Sourceurl

    FROM `gdelt-bq.full.events`
    
    WHERE (SQLDATE >= 20210101 AND SQLDATE <= 20210115) 
    
    AND (ActionGeo_CountryCode = 'HA' OR Actor1Geo_CountryCode = 'HA' OR Actor2Geo_CountryCode = 'HA')
    
    GROUP BY SQLDATE, GlobalEventID, EventCode, EventBaseCode, EventRootCode, Quadclass, AvgTone, 
    GoldSteinScale, NumMentions, Sourceurl
    
    ORDER BY SQLDATE, GlobalEventID, EventCode, EventBaseCode, EventRootCode, Quadclass, AvgTone, 
    GoldSteinScale, NumMentions, Sourceurl
    
"""

news_df = pandas_gbq.read_gbq(query, credentials=credentials)

Downloading: 100%|██████████| 1097/1097 [00:00<00:00, 2556.92rows/s]


In [13]:
news_df.head()

,GlobalEventID,SQLDATE,EventCode,EventBaseCode,EventRootCode,Quadclass,AvgTone,GoldSteinScale,NumMentions,Sourceurl
0,962423670,20210103,017,017,01,1,2.640264,0.0,7,"http://www.jamaicaobserver.com/latestews/US,_C..."
1,962423671,20210103,017,017,01,1,2.640264,0.0,1,"http://www.jamaicaobserver.com/latestews/US,_C..."
2,962423672,20210103,046,046,04,1,2.640264,7.0,7,"http://www.jamaicaobserver.com/latestews/US,_C..."
3,962423673,20210103,046,046,04,1,2.640264,7.0,1,"http://www.jamaicaobserver.com/latestews/US,_C..."
4,962423675,20210103,051,051,05,1,2.640264,3.4,2,"http://www.jamaicaobserver.com/latestews/US,_C..."


In [59]:
# function to look for the base URL for country specific sources and return filtered df

def get_articles(fips):
    
    # find all sources from chosen country
    temp_sources = country_filter[country_filter.fips == fips]
    
    # create a pattern to search for sources in query result
    source_list = temp_sources.source.to_list()
    pattern = '|'.join(source_list)
    
    # create df with results from chosen country
    articles = news_df[news_df.Sourceurl.str.contains(pattern) == True]
    
    # keep unique articles only
    articles = articles.drop_duplicates(subset=['Sourceurl'])
    
    return(articles)

In [60]:
canada_articles = get_articles('CA')

In [61]:
canada_articles.head()

,GlobalEventID,SQLDATE,EventCode,EventBaseCode,EventRootCode,Quadclass,AvgTone,GoldSteinScale,NumMentions,Sourceurl
68,962310574,20210101,010,010,01,1,3.490667,0.0,16,https://www.newswire.ca/news-releases/statemen...
295,1021737031,20210104,173,173,17,4,-4.983389,-5.0,2,https://www.thestar.com.my/news/world/2022/01/...
744,963601457,20210110,042,042,04,1,-1.134931,1.9,10,https://www.theglobeandmail.com/opinion/articl...
947,964169889,20210113,036,036,03,1,-2.027027,4.0,6,https://www.kamloopsnews.ca/news/city-region/k...
978,964200375,20210113,042,042,04,1,2.439024,1.9,10,https://montrealgazette.com/opinion/columnists...


In [81]:
# creating dictionary to hold the urls and their respective text 
link_text = {}


# function to scrape the text from articles and attach them to df
def get_article_text(df):
    
    # creates a list of URLs to use for nested function
    url_list = df['Sourceurl'].tolist()

    # function to scrape the text from article URLs
    def scraper(url):

        # using the Article function from newspaper package
        article = Article(i)
        try:
            article.download()
            article.parse()
            article.text
            link_text[i] = article.text

        # if the URL is not active, include below text    
        except:
            link_text[i] = 'URL not found.'
        return
    
    # looping through the list of URLs
    for i in url_list:
        scraper(i)
        
    # creating a df from the dict
    temp_df = pd.DataFrame(list(link_text.items()), columns = ['Sourceurl', 'article_text'])
    
    # joining the text into the existing df
    output_df = pd.merge(df, temp_df, on='Sourceurl', how='left')
    
    return(output_df)

In [82]:
final_df = get_article_text(canada_articles)

In [85]:
final_df.head()

,GlobalEventID,SQLDATE,EventCode,EventBaseCode,EventRootCode,Quadclass,AvgTone,GoldSteinScale,NumMentions,Sourceurl,article_text
0,962310574,20210101,010,010,01,1,3.490667,0.0,16,https://www.newswire.ca/news-releases/statemen...,"OTTAWA, ON, Jan. 1, 2021 /CNW/ - The Prime Min..."
1,1021737031,20210104,173,173,17,4,-4.983389,-5.0,2,https://www.thestar.com.my/news/world/2022/01/...,BOGOTA/PANAMA CITY (Reuters) - Panama authorit...
2,963601457,20210110,042,042,04,1,-1.134931,1.9,10,https://www.theglobeandmail.com/opinion/articl...,A Canadian soldier aids a senior citizen on Ma...
3,964169889,20210113,036,036,03,1,-2.027027,4.0,6,https://www.kamloopsnews.ca/news/city-region/k...,URL not found.
4,964200375,20210113,042,042,04,1,2.439024,1.9,10,https://montrealgazette.com/opinion/columnists...,"This advertisement has not loaded yet, but you..."


In [86]:
# storing the final df to use in the modeling notebook

%store final_df

Stored 'final_df' (DataFrame)


#### TODO: After the pipeline is complete, look into the Orchest tool